In [2]:
from build_netmob_data import load_subway_shp,load_netmob_gdf,find_ids_within_epsilon,get_station_data_and_permute_reshape,build_image
import os 
from os import listdir
import pickle
import torch
import numpy as np 

# Init: 
if torch.cuda.is_available():
    data_FOLDER_PATH = '../../../../data/'
    save_folder = f"{data_FOLDER_PATH}NetMob_TS/"
    netmob_data_FOLDER_PATH = f"{data_FOLDER_PATH}NetMob/"
    PATH_iris = f'{data_FOLDER_PATH}lyon_iris_shapefile/'
else:
    data_FOLDER_PATH = '../../../data/'
    save_folder = f"{data_FOLDER_PATH}NetMob_TS/"
    netmob_data_FOLDER_PATH = f"{data_FOLDER_PATH}NetMob/"
    PATH_iris = '../Data/lyon_iris_shapefile/'

if not os.path.exists(save_folder):
    os.makedirs(save_folder)


assert_transfer_mode = 'DL'
# Load Ref Subway: 
ref_subway = load_subway_shp(FOLDER_PATH = data_FOLDER_PATH)

# Parameters: size of netmob image 
step_south_north = 287  # Incremente by 287-ids when passing from south to north. 
epsilon=200 #epsilon : radius, in meter (1000m) 
# W,H = 2*(epsilon//100 + 1), 2*(epsilon//100 + 1)

# Load subway gdf adn NetMob gdf
Netmob_gdf,working_zones = load_netmob_gdf(FOLDER_PATH = netmob_data_FOLDER_PATH,
                            data_folder = PATH_iris, 
                            geojson_path = 'Lyon.geojson',
                            zones_path = 'lyon.shp')
Netmob_gdf_dropped = Netmob_gdf.drop_duplicates(subset = ['tile_id'])  # Some Doubles are exis

# Get Cell-Id within epsilon : 
result,joined = find_ids_within_epsilon(Netmob_gdf_dropped,ref_subway,epsilon=epsilon) 
maxi_nb_tile =  result.apply(lambda row: len(row.tile_id),axis=1).max()
print(f"Maximum number of NetMob Cell associated to a subway station: {maxi_nb_tile}")

''' 
Build and Save NetMob Tensor Image associated to a station 'i': 
NetMob Tensor Image: 5-th order Tensor [T]
'''

#apps = [app for app in listdir(netmob_data_FOLDER_PATH) if ((app != 'Lyon.geojson') and (not app.startswith('.'))) ]   # Avoid hidden folder and Lyon.geojson

apps = ['Uber', 'Google_Maps','Waze','Deezer','Apple_Music','Facebook','Instagram','Facebook_Messenger','WhatsApp']
Features_by_stations = []
for i in range(len(ref_subway)):
    TS_apps = []
    for app in apps: 

        metadata = pickle.load(open(f"{data_FOLDER_PATH}NetMob_tensor/{app}_metadata.pkl","rb"))
        T = torch.load(f"{data_FOLDER_PATH}NetMob_tensor/{app}.pt")  #[day, transfer_mode, Station, Tile_id, (hour,minutes)]

        stations = list(metadata.keys())
        station = stations[i]
        pos = 0 if assert_transfer_mode == metadata[station]['mode_transfer'][0] else 1
        tile_ids = np.array(metadata[station]['tile_id'])

        T_i = get_station_data_and_permute_reshape(T,i)
        TS_app = T_i[:,pos,:].float().mean(-1)
        
        TS_apps.append(TS_app)
    name_save = f"station_{station}"
    TS_apps = torch.stack(TS_apps,dim=1)
    Features_by_stations.append(TS_apps)

Features_by_stations = torch.stack(Features_by_stations,dim=1)
torch.save(Features_by_stations,f"{save_folder}NetMob_TS.pt")

metadata_ts = {'apps':apps,
               'transfer_mode': assert_transfer_mode,
               'stations': stations,
               'Format': '[T,N,C]. T:  number of timeslots; N: number of spatial units; C: number of apps'
              } 
pickle.dump(metadata_ts,open(f'{save_folder}metadata.pkl','wb'))

DriverError: ../Data/lyon_iris_shapefile/lyon.shp: No such file or directory

In [ ]:
#Load the saved image : 
netmob_T = torch.load(f"{save_folder}NetMob_TS.pt")  #[day, transfer_mode, Station, Tile_id, (hour,minutes)]
metadata_ts = pickle.load(open(f'{save_folder}metadata.pkl','rb'))
netmob_T.size()